In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar10/data_batch_1
/kaggle/input/cifar10/data_batch_2
/kaggle/input/cifar10/batches.meta
/kaggle/input/cifar10/test_batch
/kaggle/input/cifar10/data_batch_3
/kaggle/input/cifar10/data_batch_5
/kaggle/input/cifar10/data_batch_4


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [3]:
import torchvision

In [4]:
import torch.nn.functional as F

In [5]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


train_batch = {}
test_batch = unpickle('/kaggle/input/cifar10/test_batch')
for i in range(5):
    train_batch[f'batch_{i+1}'] = unpickle("/kaggle/input/cifar10/data_batch_%d"%(i + 1))

In [6]:
def get_dataset(batch_dict,test_dict):


    training_image = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'data']),torch.tensor(batch_dict['batch_2'][b'data']),torch.tensor(batch_dict['batch_3'][b'data']),torch.tensor(batch_dict['batch_4'][b'data']),torch.tensor(batch_dict['batch_5'][b'data'])),0),dtype = torch.float32)
    training_labels = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'labels']),torch.tensor(batch_dict['batch_2'][b'labels']),torch.tensor(batch_dict['batch_3'][b'labels']),torch.tensor(batch_dict['batch_4'][b'labels']),torch.tensor(batch_dict['batch_5'][b'labels']))))     

    test_images = torch.tensor(test_dict[b'data'],dtype = torch.float32)
    test_labels = torch.tensor(test_dict[b'labels'])

    training_image = training_image.view(50000,3,32,32)
    test_images = test_images.view(test_images.shape[0],3,32,32)

    return training_image,training_labels,test_images,test_labels


def make_batches(batch_size=16):
    training_image,training_labels,test_images,test_labels = get_dataset(train_batch,test_batch)
    train_dataset = TensorDataset(training_image,training_labels)
    test_dataset = TensorDataset(test_images,test_labels)
    train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
    test_loader = DataLoader(test_dataset,shuffle =True,batch_size=batch_size)

    return train_loader,test_loader

In [7]:

def training(model,criterion,optimizer,train_loader,epochs=10):
    training_loss = 0.0
    training_accuracy = 0
    for epoch in range(epochs):
        for i,(images,labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = criterion(output,labels)

            # training_loss += loss.item()
            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            _, predicted = torch.max(output.data, 1)

            training_accuracy += (predicted == labels).sum().item()
            if ((i+1)%100 == 0):
                print(f'epoch:{epoch + 1},loss:{loss.item():.4f}')
        print(f'accuracy:{training_accuracy*100/50000}')
        training_accuracy = 0    

#     torch.save('Network/model.pth',model.state_dict())

    return model

In [8]:
def evaluation(model,test_loader):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        test_loss = 0.0
        test_accuracy = 0
        for i,(images, labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            
        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network on the 10000 test images: {acc} %')

In [9]:
train_loader,_ = make_batches(128)
_,test_loader = make_batches(128)

/tmp/ipykernel_27/3786781395.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_image = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'data']),torch.tensor(batch_dict['batch_2'][b'data']),torch.tensor(batch_dict['batch_3'][b'data']),torch.tensor(batch_dict['batch_4'][b'data']),torch.tensor(batch_dict['batch_5'][b'data'])),0),dtype = torch.float32)
/tmp/ipykernel_27/3786781395.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_labels = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'labels']),torch.tensor(batch_dict['batch_2'][b'labels']),torch.tensor(batch_dict['batch_3'][b'labels']),torch.tensor(batch_dict['batch_4'][b'labels']),torch.tenso

In [10]:
class ConvBN(nn.Module):
    
    def __init__(self,input_channels,filters,strides = 1,kernel_size = 3,padding = 1,rate = 0.25,drop = False):
        super(ConvBN,self).__init__()
        self.drop = drop
        self.conv = nn.Conv2d(input_channels,filters,kernel_size = kernel_size,padding = padding,stride = strides)
        self.bnorm = nn.BatchNorm2d(filters)
#         self.dropout = nn.dropout(rate)
        
    def forward(self,inputs):
        
        output = self.conv(inputs)
        
        output = F.relu(self.bnorm(output))
        
#         if self.drop:
#             output = self.droupout(output)
            
        return output
        

In [11]:
class VGG16(nn.Module):
    
    def __init__(self):
        super(VGG16,self).__init__()
        
        self.conv1 = ConvBN(3,64)
        self.conv2 = ConvBN(64,64)
        
        self.conv3 = ConvBN(64,128)
        self.conv4 = ConvBN(128,128)
        
        self.conv5 = ConvBN(128,256)
        self.conv6 = ConvBN(256,256)
        self.conv7 = ConvBN(256,256)
        
        self.conv8 = ConvBN(256,512)
        self.conv9 = ConvBN(512,512)
        self.conv10 = ConvBN(512,512)
        
        self.conv11 = ConvBN(512,512)
        self.conv12 = ConvBN(512,512)
        self.conv13 = ConvBN(512,512)
        
        
        self.fc1 = nn.Linear(512,512)
        self.fc2 = nn.Linear(512,10)
    
        self.maxpool = nn.MaxPool2d(2,2)
        self.batchnorm = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.5)
        
        
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.maxpool(self.conv2(x))
        
        x = self.conv3(x)
        x = self.maxpool(self.conv4(x))
        
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.maxpool(self.conv7(x))
        
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.maxpool(self.conv10(x))
        
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.maxpool(self.conv13(x))
        
        x = self.dropout(x)
        
        x = x.view(x.shape[0],512*1*1)
        
        x = self.fc1(x)
        
        x = self.batchnorm(x)
        
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x 
        
        

In [12]:
model = VGG16()

In [13]:
device= 'cuda'

In [14]:
model.to(device)

VGG16(
  (conv1): ConvBN(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): ConvBN(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): ConvBN(
    (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): ConvBN(
    (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv5): ConvBN(
    (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv6): ConvB

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.1)

In [16]:
new_model = training(model,criterion,optimizer,train_loader,epochs = 15)

epoch:1,loss:1.8135
epoch:1,loss:1.6363
epoch:1,loss:1.5103
accuracy:35.178
epoch:2,loss:1.1336
epoch:2,loss:0.9601
epoch:2,loss:0.9183
accuracy:62.776
epoch:3,loss:0.6517
epoch:3,loss:0.8039
epoch:3,loss:0.8106
accuracy:74.16
epoch:4,loss:0.6506
epoch:4,loss:0.6319
epoch:4,loss:0.5263
accuracy:79.834
epoch:5,loss:0.4559
epoch:5,loss:0.3924
epoch:5,loss:0.3838
accuracy:83.894
epoch:6,loss:0.2901
epoch:6,loss:0.4023
epoch:6,loss:0.3958
accuracy:87.088
epoch:7,loss:0.1965
epoch:7,loss:0.4330
epoch:7,loss:0.4754
accuracy:89.828
epoch:8,loss:0.2278
epoch:8,loss:0.1983
epoch:8,loss:0.1943
accuracy:91.734
epoch:9,loss:0.2457
epoch:9,loss:0.1989
epoch:9,loss:0.1861
accuracy:93.286
epoch:10,loss:0.1249
epoch:10,loss:0.2883
epoch:10,loss:0.2747
accuracy:94.726
epoch:11,loss:0.0701
epoch:11,loss:0.1656
epoch:11,loss:0.1069
accuracy:95.668
epoch:12,loss:0.1309
epoch:12,loss:0.0364
epoch:12,loss:0.1030
accuracy:96.686
epoch:13,loss:0.1007
epoch:13,loss:0.0581
epoch:13,loss:0.1134
accuracy:96.982
e

In [17]:
evaluation(model,test_loader)

Accuracy of the network on the 10000 test images: 83.94 %
